```sentences_NH/engbook37``` is an input file being referenced here, but I don't have, nor do I have a script that generates it. 

Using the Vecalign script vecalign.py (partially modified) from https://github.com/thompsonb/vecalign, the code takes as input the originary files containing the sentences, the overlap files, the files containing the embeddings. The output of the alignment is a list of indexes pairs and a float number with the compute of the alignment cost.

The modification of the vecalign.py file consists in generating a .TXT file containing the output of the alignment.

In [1]:
# # Use latest conda
# !conda update conda -y
# # Create conda environment
# !conda create  --force -y --name vecalign python=3.7
# # Activate new environment
# !source `conda info --base`/etc/profile.d/conda.sh # See: https://github.com/conda/conda/issues/7980
# !conda activate vecalign
# # Install required packages
# !conda install -y -c anaconda cython
# !conda install -y -c anaconda numpy
# !pip install mcerp 

In [2]:
## generate overlaps file using the eng and lat sentences in the sentences_NH folder
# !python overlap.py -i sentences_NH/engbook37 -o sentences_NH/engoverlaps37.eng -n 10
# !python overlap.py -i sentences_NH/latbook37 -o sentences_NH/latoverlaps37.lat -n 10

#we do not use shell calls any more. We import the overlap utility and call the go function. 
#this function is normally called whenever the user performs the command above
#Inside the overlap utility the go function is a redundant copy of make_overlap(), we call that 
#without retaing go for maintainability. 
#we call make_overlap with arguments: inputfile, outputfile, numberoverlaps
#TODO: make parameters dynamic!
#   replaced by conten in next cell
# from utils import overlap
# overlap.make_overlap('sentences_NH/engbook37', 'sentences_NH/engoverlaps37.eng', 10)
# overlap.make_overlap('sentences_NH/latbook37', 'sentences_NH/latoverlaps37.lat', 10)

In [3]:
from utils import overlap
from utils.dp_core import *
from setuptools import distutils  #TODO required?
from tqdm import tqdm
import os
from utils import vecalign

#TODO: make parameters dynamic
#TODO: discuss with Laura; where are the source files?
#       the target file is created by the overlap utility; does not need to exist.
#       BUT, we need to know what it is used for!

for i in tqdm(range(1,38)): 
    overlap.make_overlap(source = 'data/intermediate/books/engbook'+str(i), 
                     target = 'data/intermediate/books/engoverlaps'+str(i)+'.eng', 
                     num_overlaps = 10
                     )
    overlap.make_overlap(source = 'data/intermediate/books/latbook'+str(i), 
                     target = 'data/intermediate/books/latoverlaps'+str(i)+'.lat', 
                     num_overlaps = 10
                     )

100%|██████████| 37/37 [00:02<00:00, 14.88it/s]


In [4]:
# pip install sentence_transformers

In [5]:
## import the S-BERT Python library sentence_transformer
import sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE')

c:\Workdir\MyApps\Python_VENV\geoparsing_naturalhistory-main\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Workdir\MyApps\Python_VENV\geoparsing_naturalhistory-main\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
##CONFIG:
#when set to True this will redo the labse files even if they exists.
#when set to False, LABSE files will be retained, a new model won't be made.
renew_labse_files = False

basedir = 'data/intermediate/books'
lang1 = 'eng'
lang2 = 'lat'

files = os.listdir(basedir)
# Filtering only the files.
files = [f for f in files if os.path.isfile(basedir+'/'+f) and f.strip().endswith(lang1)]

#prepare a typle of embeddings with per tuple the Source and Target
all_embeddings = []

# for i in range(1,38):    
for i in files:    
    print(i)
    #skip book 1: this is only the preface. Nothing expected there. 
    if i.endswith('s1.eng'): 
        continue
    
    ## get the eng overlap file
    # path = "/Users/u0154817/OneDrive - KU Leuven/Desktop/sentences_NH/engoverlaps"+str(i)+'.eng'
    path = basedir+'/'+i
    print(path)
    with open(path, "r", encoding="utf8") as engfile:
        overlap_sentences_eng = []
        for line in engfile: ## create a list of the overlap eng sentences
            overlap_sentences_eng.append(line)
    
    ## get the lat overlap file
    path = path.replace('engoverlaps', 'latoverlaps')
    path = path.replace('.eng', '.lat')
    print(path)
    with open(path, "r", encoding="utf8") as latfile:
        overlap_sentences_lat = []
        for line in latfile:  ## create a list of the overlap lat sentences
            overlap_sentences_lat.append(line)
    ## generate the name of the LABSE files: 
    j = i.split('.')[0]
    j = j.replace('engoverlaps', '').strip()
    src_embedded_filename = 'overlaps.'+lang1+'.'+j+'.LaBSE.emb'
    tgt_embedded_filename = 'overlaps.'+lang2+'.'+j+'.LaBSE.emb'
    # if the files exist, do not override them:
    engoverlap_missing = not os.path.exists('data/intermediate/labse_embeddings/overlaps.eng.'+str(j)+'.LaBSE.emb')
    latoverlap_missing = not os.path.exists('data/intermediate/labse_embeddings/overlaps.lat.'+str(j)+'.LaBSE.emb')
    ## generate the embeddings of the eng and lat overlap sentences (conditionally)
    if renew_labse_files or engoverlap_missing: 
        print(f'making english embeddings: {len(overlap_sentences_eng)}')
        overlaps_embeddings_eng_LaBSE = model.encode(overlap_sentences_eng)
        overlaps_embeddings_eng_LaBSE.tofile('data/intermediate/labse_embeddings/overlaps.eng.'+str(j)+'.LaBSE.emb')
    else:
        print('Skipping creation of '+src_embedded_filename+'. Embeddings exist and cell is in no-ovverride mode.')
    if renew_labse_files or latoverlap_missing:
        print(f'making latin embeddings: {len(overlap_sentences_lat)}')
        overlaps_embeddings_lat_LaBSE = model.encode(overlap_sentences_lat)
        overlaps_embeddings_lat_LaBSE.tofile('data/intermediate/labse_embeddings/overlaps.lat.'+str(j)+'.LaBSE.emb')
    else:
        print('Skipping creation of '+tgt_embedded_filename+'. Embeddings exist and cell is in no-ovverride mode.')



engoverlaps1.eng
engoverlaps10.eng
data/intermediate/books/engoverlaps10.eng
data/intermediate/books/latoverlaps10.lat
Skipping creation of overlaps.eng.10.LaBSE.emb. Embeddings exist and cell is in no-ovverride mode.
Skipping creation of overlaps.lat.10.LaBSE.emb. Embeddings exist and cell is in no-ovverride mode.
engoverlaps11.eng
data/intermediate/books/engoverlaps11.eng
data/intermediate/books/latoverlaps11.lat
making english embeddings: 9276
making latin embeddings: 10026
engoverlaps12.eng
data/intermediate/books/engoverlaps12.eng
data/intermediate/books/latoverlaps12.lat
making english embeddings: 4211
making latin embeddings: 4536
engoverlaps13.eng
data/intermediate/books/engoverlaps13.eng
data/intermediate/books/latoverlaps13.lat
making english embeddings: 3771
making latin embeddings: 4156
engoverlaps14.eng
data/intermediate/books/engoverlaps14.eng
data/intermediate/books/latoverlaps14.lat
making english embeddings: 3786
making latin embeddings: 4156


In [ ]:
import os 
dir = os.getcwd()
os.path.join(dir, 'data', 'intermediate', 'books', 'engbook10')

In [ ]:
##TODO: here you'll need to create a forloop that iterates over all labse_embeddings files. 
# categorize them in latin and english ambeddings
# link them by the number in their name
# then run the linked versions in the vecalign_custom method.
i = os.path.join(dir, 'data', 'intermediate', 'books', 'engbook10')
path = os.path.join(dir, 'data', 'intermediate', 'books', 'latbook10')
src_embedded_filename = 'data/intermediate/labse_embeddings/overlaps.eng.10.LaBSE.emb'
tgt_embedded_filename = 'data/intermediate/labse_embeddings/overlaps.lat.10.LaBSE.emb'


vecalign.vecalign_custom(
        src=i, 
        tgt=path, 
        src_embed=('data/intermediate/books/engoverlaps10.eng', src_embedded_filename), 
        tgt_embed=('data/intermediate/books/latoverlaps10.lat', tgt_embedded_filename) 
    )

In [ ]:
# ## calculate the sentence similarity using the sentence files, the overlap files, and the sentence embeddings
# !python vecalign.py --alignment_max_size 8 --src sentences_NH/engbook37 --tgt sentences_NH/latbook37 \
#    --src_embed sentences_NH/engoverlaps37.eng sentences_NH/overlaps.eng.37.LaBSE.emb  \
#    --tgt_embed sentences_NH/latoverlaps37.lat sentences_NH/overlaps.lat.37.LaBSE.emb

In [ ]:
# vecalign(
#     src='sentences_NH/engbook37', 
#     tgt='sentences_NH/latbook37', 
#       src_embed and tgt_embed have to become tuples: 
#       first is a text file, second is a binary embeddings file
#     src_embed='sentences_NH/engoverlaps37.eng sentences_NH/overlaps.eng.37.LaBSE.emb', 
#     tgt_embed='sentences_NH/latoverlaps37.lat sentences_NH/overlaps.lat.37.LaBSE.emb'
#     )